In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, confusion_matrix, classification_report, accuracy_score
import seaborn as sns
import glob
import tensorflow as tf
from tensorflow.keras import layers, optimizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

%matplotlib inline
# For Filtering the warnings
import warnings
warnings.filterwarnings('ignore')

In [ ]:
data=pd.read_csv('kidney_disease.csv')

In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Data columns (total 26 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              400 non-null    int64  
 1   age             391 non-null    float64
 2   bp              388 non-null    float64
 3   sg              353 non-null    float64
 4   al              354 non-null    float64
 5   su              351 non-null    float64
 6   rbc             248 non-null    object 
 7   pc              335 non-null    object 
 8   pcc             396 non-null    object 
 9   ba              396 non-null    object 
 10  bgr             356 non-null    float64
 11  bu              381 non-null    float64
 12  sc              383 non-null    float64
 13  sod             313 non-null    float64
 14  pot             312 non-null    float64
 15  hemo            348 non-null    float64
 16  pcv             330 non-null    object 
 17  wc              295 non-null    obj

In [ ]:
data.classification=data.classification.replace("ckd\t","ckd")
data.classification.unique()

array(['ckd', 'notckd'], dtype=object)

In [ ]:
data.drop('id', axis = 1, inplace = True)
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,ckd
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,ckd
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,ckd
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,ckd
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,ckd


In [ ]:
data['classification'] = data['classification'].replace(['ckd','notckd'], [1,0])
data.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
0,48.0,80.0,1.020,1.0,0.0,NaN,normal,notpresent,notpresent,121.0,...,44,7800,5.2,yes,yes,no,good,no,no,1
1,7.0,50.0,1.020,4.0,0.0,NaN,normal,notpresent,notpresent,NaN,...,38,6000,NaN,no,no,no,good,no,no,1
2,62.0,80.0,1.010,2.0,3.0,normal,normal,notpresent,notpresent,423.0,...,31,7500,NaN,no,yes,no,poor,no,yes,1
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,1
4,51.0,80.0,1.010,2.0,0.0,normal,normal,notpresent,notpresent,106.0,...,35,7300,4.6,no,no,no,good,no,no,1


In [ ]:
df = data.dropna(axis = 0)
print(f"Before dropping all NaN values: {data.shape}")
print(f"After dropping all NaN values: {df.shape}")
df.head()

Before dropping all NaN values: (400, 25)
After dropping all NaN values: (158, 25)


,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,48.0,70.0,1.005,4.0,0.0,normal,abnormal,present,notpresent,117.0,...,32,6700,3.9,yes,no,no,poor,yes,yes,1
9,53.0,90.0,1.020,2.0,0.0,abnormal,abnormal,present,notpresent,70.0,...,29,12100,3.7,yes,yes,no,poor,no,yes,1
11,63.0,70.0,1.010,3.0,0.0,abnormal,abnormal,present,notpresent,380.0,...,32,4500,3.8,yes,yes,no,poor,yes,no,1
14,68.0,80.0,1.010,3.0,2.0,normal,abnormal,present,present,157.0,...,16,11000,2.6,yes,yes,yes,poor,yes,no,1
20,61.0,80.0,1.015,2.0,0.0,abnormal,abnormal,notpresent,notpresent,173.0,...,24,9200,3.2,yes,yes,yes,poor,yes,yes,1


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             158 non-null    float64
 1   bp              158 non-null    float64
 2   sg              158 non-null    float64
 3   al              158 non-null    float64
 4   su              158 non-null    float64
 5   rbc             158 non-null    object 
 6   pc              158 non-null    object 
 7   pcc             158 non-null    object 
 8   ba              158 non-null    object 
 9   bgr             158 non-null    float64
 10  bu              158 non-null    float64
 11  sc              158 non-null    float64
 12  sod             158 non-null    float64
 13  pot             158 non-null    float64
 14  hemo            158 non-null    float64
 15  pcv             158 non-null    object 
 16  wc              158 non-null    object 
 17  rc              158 non-null    obj

In [ ]:
df['pcv']=df['pcv'].astype(int)
df['wc']=df['wc'].astype(int)
df['rc']=df['rc'].astype(float)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 158 entries, 3 to 399
Data columns (total 25 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             158 non-null    float64
 1   bp              158 non-null    float64
 2   sg              158 non-null    float64
 3   al              158 non-null    float64
 4   su              158 non-null    float64
 5   rbc             158 non-null    object 
 6   pc              158 non-null    object 
 7   pcc             158 non-null    object 
 8   ba              158 non-null    object 
 9   bgr             158 non-null    float64
 10  bu              158 non-null    float64
 11  sc              158 non-null    float64
 12  sod             158 non-null    float64
 13  pot             158 non-null    float64
 14  hemo            158 non-null    float64
 15  pcv             158 non-null    int64  
 16  wc              158 non-null    int64  
 17  rc              158 non-null    flo

In [ ]:
dictonary = {
        "rbc": {
        "abnormal":1,
        "normal": 0,
    },
        "pc":{
        "abnormal":1,
        "normal": 0,
    },
        "pcc":{
        "present":1,
        "notpresent":0,
    },
        "ba":{
        "notpresent":0,
        "present": 1,
    },
        "htn":{
        "yes":1,
        "no": 0,
    },
        "dm":{
        "yes":1,
        "no":0,
    },
        "cad":{
        "yes":1,
        "no": 0,
    },
        "appet":{
        "good":1,
        "poor": 0,
    },
        "pe":{
        "yes":1,
        "no":0,
    },
        "ane":{
        "yes":1,
        "no":0,
    }
}

In [ ]:
df=df.replace(dictonary)

In [ ]:
df.head()

,age,bp,sg,al,su,rbc,pc,pcc,ba,bgr,...,pcv,wc,rc,htn,dm,cad,appet,pe,ane,classification
3,48.0,70.0,1.005,4.0,0.0,0,1,1,0,117.0,...,32,6700,3.9,1,0,0,0,1,1,1
9,53.0,90.0,1.020,2.0,0.0,1,1,1,0,70.0,...,29,12100,3.7,1,1,0,0,0,1,1
11,63.0,70.0,1.010,3.0,0.0,1,1,1,0,380.0,...,32,4500,3.8,1,1,0,0,1,0,1
14,68.0,80.0,1.010,3.0,2.0,0,1,1,1,157.0,...,16,11000,2.6,1,1,1,0,1,0,1
20,61.0,80.0,1.015,2.0,0.0,1,1,0,0,173.0,...,24,9200,3.2,1,1,1,0,1,1,1


In [ ]:
countNoDisease = len(df[df['classification'] == 0])
countHaveDisease = len(df[df['classification'] == 1])
print("Percentage of Patients Haven't CKD Disease: {:.2f}%".format((countNoDisease / (len(df['classification']))*100)))
print("Percentage of Patients Have CKD Disease: {:.2f}%".format((countHaveDisease / (len(df['classification']))*100)))


Percentage of Patients Haven't CKD Disease: 72.78%
Percentage of Patients Have CKD Disease: 27.22%


In [ ]:
from sklearn.model_selection import train_test_split
X = df.drop(['classification', 'sg', 'appet', 'rc', 'pcv', 'hemo', 'sod'], axis = 1)
y = df['classification']

In [ ]:
X.columns

Index(['age', 'bp', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr', 'bu', 'sc',
       'pot', 'wc', 'htn', 'dm', 'cad', 'pe', 'ane'],
      dtype='object')

In [ ]:
!pip install shap
import shap
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import GRU, Dense
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

model = Sequential([
    GRU(64, input_shape=(X_train.shape[1], 1)),
    Dense(3, activation='softmax')  # Assuming you have 3 classes
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(X_train[:, :, np.newaxis], y_train, epochs=5, batch_size=32, validation_split=0.2)

explainer = shap.GradientExplainer(model, X_train[:, :, np.newaxis])
shap_values = explainer.shap_values(X_test[:, :, np.newaxis])

print("Shape of shap_values:", shap_values[0].shape)

y_pred = np.argmax(model.predict(X_test[:, :, np.newaxis]), axis=1)

accuracy_gru = accuracy_score(y_test, y_pred)
print(f'Accuracy(GRU): {accuracy_gru:.2f}')

Epoch 1/5
4/4 [==============================] - 6s 178ms/step - loss: 1.1130 - accuracy: 0.3300 - val_loss: 1.0518 - val_accuracy: 0.8846
Epoch 2/5
4/4 [==============================] - 0s 19ms/step - loss: 1.0201 - accuracy: 0.9400 - val_loss: 0.9813 - val_accuracy: 0.8846
Epoch 3/5
4/4 [==============================] - 0s 18ms/step - loss: 0.9343 - accuracy: 0.9500 - val_loss: 0.9133 - val_accuracy: 0.8846
Epoch 4/5
4/4 [==============================] - 0s 25ms/step - loss: 0.8507 - accuracy: 0.9600 - val_loss: 0.8395 - val_accuracy: 0.8846
Epoch 5/5
4/4 [==============================] - 0s 19ms/step - loss: 0.7625 - accuracy: 0.9600 - val_loss: 0.7579 - val_accuracy: 0.8846
Shape of shap_values: (18, 1, 3)
1/1 [==============================] - 0s 379ms/step
Accuracy(GRU): 0.97


In [ ]:
import numpy as np

# Assuming your original input_data with missing values
input_data = (24,100,	2,0,1,0,1,0,136,60,1.9,3.7,9600,1,1,0,0,1)

# Change the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Impute missing values with the mean of the non-missing values
input_data_as_numpy_array = np.nan_to_num(input_data_as_numpy_array, nan=np.nanmean(input_data_as_numpy_array))

# Reshape the numpy array to match the expected shape for XGBoost prediction
input_data_reshaped = input_data_as_numpy_array.reshape(1, 1, -1)

# Print the number of features in both datasets
print("Number of features in X_train_reshaped:", X_train_reshaped.shape[2])
print("Number of features in input_data_reshaped:", input_data_reshaped.shape[2])

# Assuming you have already defined and trained your GRU model
prediction = model.predict(input_data_reshaped)

if prediction[0, 0] <= 0.5:
    print("The person does not have CKD disease.")
else:
    print("The person has CKD disease.")
    print('Accuracy:', train_accuracy)

Number of features in X_train_reshaped: 18
Number of features in input_data_reshaped: 18


InvalidArgumentError: Graph execution error:

Detected at node while/BiasAdd defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/usr/local/lib/python3.10/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.10/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.10/dist-packages/ipykernel/kernelapp.py", line 619, in start

  File "/usr/local/lib/python3.10/dist-packages/tornado/platform/asyncio.py", line 195, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

  File "/usr/lib/python3.10/asyncio/base_events.py", line 1909, in _run_once

  File "/usr/lib/python3.10/asyncio/events.py", line 80, in _run

  File "/usr/local/lib/python3.10/dist-packages/tornado/ioloop.py", line 685, in <lambda>

  File "/usr/local/lib/python3.10/dist

Saving Trained Model


In [ ]:
import pickle

In [ ]:
filename='trainedmodel.sav'
pickle.dump(model,open(filename,'wb'))

In [ ]:
#loading the saved model

In [ ]:
loaded_model=pickle.load(open('trainedmodel.sav','rb'))

In [ ]:
import numpy as np

# Assuming your original input_data with missing values
input_data = (24,100,	2,	0,	1,	0,	1,	0,	136,	60,	1.9,	3.7,	9600,	1,	1,	0,	0,	1)

# Change the input data to a numpy array
input_data_as_numpy_array = np.asarray(input_data)

# Impute missing values with the mean of the non-missing values
input_data_as_numpy_array = np.nan_to_num(input_data_as_numpy_array, nan=np.nanmean(input_data_as_numpy_array))

# Reshape the numpy array to match the expected shape for XGBoost prediction
input_data_reshaped = input_data_as_numpy_array.reshape(1, 1, -1)

# Assuming you have already defined and trained your GRU model
prediction =loaded_model.predict(input_data_reshaped)

if prediction[0, 0] <= 0.5:
    print("The person does not have CKD disease.")
else:
    print("The person has CKD disease.")
    print('Accuracy:', train_accuracy)


Number of features in X_train_reshaped: 18
Number of features in input_data_reshaped: 18
1/1 [==============================] - 0s 422ms/step
The person has CKD disease.
Accuracy: 0.9841269841269841
